In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Model is based on paper https://www.xajzkjdx.cn/gallery/64-nov2020.pdf to classify 23 skin diseases images dataset , please go through code 

In [ ]:
import os

for dirpath, dirnames, filenames in os.walk('/kaggle/input'):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
train_dir='../input/dermnet/train/'
test_dir='../input/dermnet/test/'

In [ ]:
import pathlib
import numpy as np
data_dir=pathlib.Path(train_dir)
class_names=np.array(sorted(item.name for item in data_dir.glob("*")))
print(class_names)
print("\n")
print(f'In total we have {len(class_names)} classes')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
IMG_SIZE=(224,224)
train_data=tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                              image_size=IMG_SIZE,
                                                              label_mode='categorical',
                                                               batch_size=16
                                                            
                                                              )
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                             image_size=IMG_SIZE,
                                                             label_mode='categorical',
                                                              batch_size=16
                                                            
                                                             )

In [ ]:
train_data.class_names

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
data_augmentation = keras.Sequential([
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
    preprocessing.RandomFlip('horizontal')
], name ="data_augmentation")

In [ ]:

base_model =tf.keras.applications.inception_v3.InceptionV3(
                                                           include_top=False,
                                                           weights=None
                                                           )
    

base_model.trainable = True
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")
x=data_augmentation(inputs)
x = base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name="GloPool")(x)
outputs = tf.keras.layers.Dense(23, activation="softmax", name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=["accuracy"])

In [ ]:
model_0.fit(train_data,
           epochs=60,
           steps_per_epoch=len(train_data),
           validation_data=test_data,
           validation_steps=int(0.25*len(test_data)))

In [ ]:
model_0.evaluate(test_data)